In [ ]:
import argparse

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

from lang_sam import LangSAM

In [ ]:
model = LangSAM()
image_pil = Image.open("sample_input_images/chair.jpg")
masks, boxes, phrases, logits = model.predict(image_pil.convert("RGB"), "chair")
masks=masks.numpy()[0]
boxes=[int(x) for x in boxes.tolist()[0]]


In [ ]:

image_array=np.array(image_pil)/255
masked_image_array=np.copy(image_array)
white_image_array=np.ones_like(masked_image_array,dtype=np.float32)

bounding_box=white_image_array*(1-masks[:, :, np.newaxis])+masked_image_array*masks[:, :, np.newaxis]
bounding_box=bounding_box[int(boxes[1]):int(boxes[3]),int(boxes[0]):int(boxes[2]),:]

plt.imshow(bounding_box)
plt.axis('off')
plt.savefig("test.png", bbox_inches='tight')


In [ ]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16, variant="fp16"
)


In [ ]:
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")


In [ ]:
generator = torch.Generator("cuda").manual_seed(92)

prompt = "concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k"
image = pipeline(prompt=prompt, image=init_image, mask_image=mask_image, generator=generator).images[0]
make_image_grid([init_image, mask_image, image], rows=1, cols=3)